**Ex *6*- Statistics Lab**

The Solution of the exercise in the bottom as Q1 is continuing of Lab 5.

Erel Rojanski, David Guedalia

In [1]:
# Add a check if we run in google colab or locally in jupyter notebook
run_in_colab = False
if 'google.colab' in str(get_ipython()):
    run_in_colab = True
    print('Running on CoLab')
else:
    print('Running locally on Jupyter')



Running on CoLab


In [2]:
import sys
import numpy as np  # a module for working with numerical array 
import pandas as pd  # a module for working with data-frames
import statsmodels.api as sm  # a module for statistical modelling (e.g. regression analysis)
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from heapq import nlargest
from sklearn.preprocessing import normalize



/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


First, we need to mount our local drive to the colab network

In [3]:
# Mount drive in google colab
if run_in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
else:  # Set local path 
    data_path = "C:/Users/Or Zuk/Google Drive/HUJI/Teaching/Lab_52568/Data/Elections/"  


Mounted at /content/drive


In [111]:
# Loading elections 2021 elections data:
if run_in_colab:
    from google.colab import files
    uploaded = files.upload()

Saving eshkol_hevrati_calcali.xlsx to eshkol_hevrati_calcali (4).xlsx
Saving votes per ballot 2021.csv to votes per ballot 2021 (3).csv
Saving votes per city 2020.csv to votes per city 2020 (2).csv
Saving votes per city 2021.csv to votes per city 2021 (4).csv


In [115]:
# Loading elections 2021 elections data:
import io

data_type = "city" # "ballot"  # city
if run_in_colab:
    if data_type == "ballot":
      df_2021_raw = pd.read_csv(io.BytesIO(uploaded['votes per city 2021.csv']),  encoding = 'iso-8859-8', index_col='שם ישוב')
    else:
      df_2021_raw = pd.read_csv(io.BytesIO(uploaded['votes per city 2021.csv']),  encoding = 'iso-8859-8', index_col='שם ישוב')
      df_2020_raw = pd.read_csv(io.BytesIO(uploaded['votes per city 2020.csv']),  encoding = 'iso-8859-8', index_col='שם ישוב')
else:  # read local file
    if data_type == "ballot":
        df_2021_raw = pd.read_csv(data_path + 'votes per ballot 2021.csv',  encoding = 'iso-8859-8', index_col='שם ישוב')
    else:
        df_2021_raw = pd.read_csv(data_path + 'votes per city 2021.csv',  encoding = 'iso-8859-8', index_col='שם ישוב')


df_2021 = df_2021_raw.drop('סמל ועדה', axis=1)
df_2021 = df_2021[df_2021.columns[5:-1]] # removing "metadata" columns
df_2020 = df_2020_raw.drop('סמל ועדה', axis=1)
df_2020 = df_2020[df_2020.columns[5:-1]] # removing "metadata" columns


In [ ]:

# df_2021.reset_index(inplace=True)
# df_2021.style.set_properties(**{'text-align': 'left'})
# df_2021.head()

# df_2020.style.set_properties(**{'text-align': 'left'})
# df_2020.head()
# Compute votes share only for parties above a certain threshold 
def parties_votes_percents(df, thresh):
    par = df.sum().div(df.sum().sum()).sort_values(ascending=False)
    return par[par > thresh]
print(df_2021.columns)
print(df_2020.columns)

total_votes_2021 = parties_votes_percents(df_2021, 0.035)  # total votes for each party
names_2021 = total_votes.keys() # largest parties
total_votes_2020 = parties_votes_percents(df_2020, 0.035)  # total votes for each party
names_2020 = total_votes.keys() # largest parties
# Data with only the largest parties:
df_2021 = df_2021[names]
df_cities = df_2021


## Question 1: 

A function that calculates the new design matrix of size C * (K + C) and returns it




In [8]:
# gets df of size C * K and return s size c * (K+C) when X[i][k+i] = sigma{1<j<k}(df[i][j])
def enlarge_df(df): 
    sums = np.sum(df_cities, axis=1) # X[i][k+i] = sigma{1<j<k}(df[i][j])
    fill_mat = np.zeros((df.shape[0], df.shape[0])).astype(int)
    np.fill_diagonal(fill_mat, sums) # C * C mat with diagonal of sigma
    fill_df = pd.DataFrame(fill_mat, index=df.index.tolist()) # setting matching indexes
    full_df = pd.concat([df, fill_df], axis=1, join="inner") # connecting the df's
    return full_df


Regression function from lab 4:


In [9]:
# Correct for voting turnout in cities/ballots using linear regression
def regression_turnout_correction(df, turnout):
    # Preprocess: remove cities/ballots with zero votes:
    good_inds = np.where(np.array(turnout) > 0)
    if np.min(turnout) == 0:
      print("Removed cities with no votes in simulation")
    df = df.iloc[good_inds]
    turnout = turnout.iloc[good_inds]

    p = df.sum().div(df.sum().sum())  # votes without correction
    bzb = df.sum(axis=1) / turnout # can also be read from outside
    model = sm.OLS(bzb, df).fit() # linear regression WITHOUT intercept
    q_hat = p * model.params  # M.coef_ # beta_inv
    q_hat = q_hat / q_hat.sum()  # Normalize
    return [p, q_hat, model.params] # beta_inv

New regression function that works with 2 parameters:

In [10]:

# Correct for voting turnout in cities/ballots using linear regression
def regression_turnout_correction_2params(df, turnout):
    
    # Preprocess: remove cities/ballots with zero votes:
    good_inds = np.where(np.array(turnout) > 0)
    if np.min(turnout) == 0:
      print("Removed cities with no votes in simulation")
    df = df.iloc[good_inds]
    turnout = turnout.iloc[good_inds]
    df_big = enlarge_df(df)
    p = df.sum().div(df.sum().sum())  # votes without correction
    bzb = df.sum(axis=1) / turnout # can also be read from outside
    model = sm.OLS(bzb, df_big).fit() # linear regression WITHOUT intercept

    big_params = model.params
    beta = big_params[:df_big.shape[1] - df_big.shape[0]]
    alpha = big_params[df_big.shape[1] - df_big.shape[0]:]
    beta_mat = np.tile(beta, (alpha.shape[0], 1))
    alpha_mat = np.tile(alpha, (beta.shape[0], 1)).T
    final_params = beta_mat + alpha_mat
    n_hat_tilde = df * final_params
    q_hat = np.sum(n_hat_tilde, axis=0)
    q_hat = q_hat / np.sum(q_hat)
    return [p, q_hat, final_params] # beta_inv



## Question 2: 
#### Repeat the simulations same as in lab 3, and add to the graphs the estimator with the new fixing parameter




**From** lab 4: 
Function that recieves the  $\tilde{N}$ and V matrices and samples binomial votes.

In [11]:
def sim_data(N,V):
  new_df = pd.DataFrame(np.random.binomial(N,V))
  new_df.index = df_2021.index
  v = new_df.sum(axis=1)/ N.sum(axis=1)
  return new_df, v

#### Run simulations and apply corrections

Computing  $\tilde{N}$ and $V_i$ for the 3 simulations:

In [ ]:
# Nij~ :
N_tilde = np.array(df_2021.mul(df_2021_raw['בזב'].sum()).div(df_2021_raw['כשרים'].sum()),dtype='int32')
print(N_tilde.shape)
print(df_2021.shape)
# 3 types of Vi 
v = df_2021_raw['כשרים'].div(df_2021_raw['בזב'])
u = list(np.random.permutation(0.2 + np.array(range(13)) * 0.05))
#u = [0.6,0.65,0.75,0.5,0.4,0.35,0.8,0.55,0.45,0.25,0.3,0.7,0.2]
#u_random = random.sample(u, len(u))

dim = df_2021.shape
# converting them to the desired shape:
V_v = np.tile(np.array([v]).transpose(), (1, len(names)))
V_u = np.tile(np.array(u), (dim[0],1))
#V_u_random = np.tile(np.array(u_random), (dim[0],1))

V_u_random = V_u.copy()
for i in range(df_2021.shape[0]):
  V_u_random[i,] = np.random.permutation(u)
#V_u_random = np.tile(np.array(u_random), (dim[0],1))

A function that calculates the $p$ and the  $\hat{q}$  (after fixing data to 100% votes with $v_i= \frac{kosher_i}{bzb_i}$)

In [13]:
def turnout_correction(df, fix_param):
  p = parties_votes_percents(df, 0) 
  q_hat = parties_votes_percents(df.div(fix_param, axis = "rows"), 0)
  return q_hat


Multiple simulations function:

In [14]:
def simulations(iters,N,V):
  p_mat = np.zeros([iters, len(names)])
  q_hat = np.zeros([iters, len(names)]) 
  q_hat_reg = np.zeros([iters, len(names)]) 

  q_hat_reg_new = np.zeros([iters, len(names)]) 
  p_mat_new = np.zeros([iters, len(names)])


  # in every simulation we calculate p and q_hat (after fixing the data)
  for i in range(0,iters):
    new_df, v = sim_data(N,V)
    q_hat[i,] = turnout_correction(new_df, v)
    p_mat[i,], q_hat_reg[i,] = regression_turnout_correction(new_df, v)[0:2]
    p_mat_new[i,], q_hat_reg_new[i,] = regression_turnout_correction_2params(new_df, v)[0:2]

  return [p_mat, q_hat, q_hat_reg, p_mat_new, q_hat_reg_new]

 Run 50 simulations:

In [16]:
iters = 1 # number of simulations
q_real = N_tilde.sum(axis=0)/(N_tilde.sum())
sim_v = simulations(iters, N_tilde, V_v)
mean_p_v = np.nanmean(sim_v[0],axis=0)
mean_q_v = np.nanmean(sim_v[1],axis=0)
mean_q_v_beta = np.nanmean(sim_v[2],axis=0)
std_q_v = np.std(sim_v[1],axis=0) # Compute also standard deviations 
std_q_v_beta = np.std(sim_v[2],axis=0)

mean_p_v2 = np.nanmean(sim_v[3],axis=0)
mean_q_v_beta2 = np.nanmean(sim_v[4],axis=0)
std_q_v_beta2 = np.std(sim_v[4],axis=0)


sim_u = simulations(iters, N_tilde, V_u)
mean_p_u = sim_u[0].mean(axis=0) 
mean_q_u = sim_u[1].mean(axis=0) 
mean_q_u_beta = sim_u[2].mean(axis=0)
std_q_u = sim_u[1].std(axis=0) 
std_q_u_beta = sim_u[2].std(axis=0)

mean_p_u2 = sim_u[3].mean(axis=0) 
mean_q_u_beta2 = sim_u[4].mean(axis=0)
std_q_u_beta2 = sim_u[4].std(axis=0)


sim_u_random = simulations(iters, N_tilde, V_u_random)
mean_p_u_random = sim_u_random[0].mean(axis=0)
mean_q_u_random = sim_u_random[1].mean(axis=0)
mean_q_u_random_beta = sim_u_random[2].mean(axis=0)
std_q_u_random = sim_u_random[1].std(axis=0)
std_q_u_random_beta = sim_u_random[2].std(axis=0)

mean_p_u_random = sim_u_random[3].mean(axis=0)
mean_q_u_random_beta2 = sim_u_random[4].mean(axis=0)
std_q_u_random_beta2 = sim_u_random[4].std(axis=0)




## Bar-plots showing the results

In [ ]:
# Bar plot for a party, only showing values above a certain threshold after fixing:
def party_bar_fix(q_real, q_hat, q_hat_beta, q_hat_std, q_hat_beta_std, p, q_hat_beta2, q_hat_beta_std2,):
    width = 0.15  # set column width 
    n = len(total_votes)

    rev_names = [name[::-1] for name in list(names)]
    fig, ax = plt.subplots(figsize=(10, 4))  # plt.subplots()

    q_real_bar = ax.bar(np.arange(n), list(q_real), width, color='b')
    q_hat_bar = ax.bar(np.arange(n)+width, q_hat, width, color='r', yerr = q_hat_std, capsize=5)
    q_hat_beta_bar = ax.bar(np.arange(n)+2*width, q_hat_beta, width, color='y', yerr = q_hat_beta_std, capsize=5)
    p_bar = ax.bar(np.arange(n)+3*width, p, width, color='g')
    q_hat_beta_bar_new = ax.bar(np.arange(n)+4*width, q_hat_beta2, width, color='brown', yerr = q_hat_beta_std2, capsize=5)


    ax.set_ylabel('Votes percent')
    ax.set_xlabel('Parties Names')
    ax.set_title('Votes percent per party - simultions')
    ax.set_xticks(np.arange(n))
    ax.set_xticklabels(rev_names, rotation=90)
    ax.legend((q_real_bar, q_hat_bar, q_hat_beta_bar, p_bar, q_hat_beta_bar_new), ('q real', 'q hat', 'q hat beta', 'p', 'q hat beta 2 params'))
    plt.show()

    return fig, ax

party_bar_fix(q_real, mean_q_v, mean_q_v_beta, std_q_v, std_q_v_beta, mean_p_v, mean_q_v_beta2, std_q_v_beta2)
party_bar_fix(q_real, mean_q_u, mean_q_u_beta, std_q_u, std_q_u_beta, mean_p_u, mean_q_u_beta2, std_q_u_beta2)
party_bar_fix(q_real, mean_q_u_random, mean_q_u_random_beta, std_q_u_random, std_q_u_random_beta, mean_p_u_random, mean_q_u_random_beta2, std_q_u_random_beta2)

In the first case, it is very clear that the 2 params regression is better, it doesnt have any negative values that are incorrect and it is much closer to the q_real and q_hat. its disadvantage is that it has a higher std.

In the second case, its a hard choice because the are both simillar in how much they are close but the 2 params regression has a lower std.

In the third case it is also pretty clear that the 2 params regression is a better fit.

In [18]:
if   run_in_colab:
    df_2021_raw = pd.read_csv(io.BytesIO(uploaded['votes per ballot 2021.csv']),  encoding = 'iso-8859-8', index_col='שם ישוב')

df_2021_raw = df_2021_raw[df_2021_raw.index != 'מעטפות חיצוניות']
df_2021 = df_2021_raw
df_2021 = df_2021[df_2021.columns[10:-1]] # removing "metadata" columns

names_passed = total_votes.keys()
df_cities = df_2021

df_2021_passed = df_2021[names_passed]

**Question** **3**:
**Section A**

In [ ]:
# PCA of the two main components.
pca = PCA(n_components=2)
df_2021_transform = df_2021.T

# PCA fit.
pca_a = pca.fit_transform(df_2021_transform)
fig, ax = plt.subplots(figsize=(20,20))
first = pca_a[:,0]
second = pca_a[:,1]

ax.scatter(first, second)
ax.set_xlabel('first component pc', fontsize=16)
ax.set_ylabel('second component pc', fontsize=16)
ax.set_title('Only two dimension pca', fontsize=16)
names = df_2021.columns

numner_of_parties = len(names)
correction_names = [x[::-1] for x in names]

for i in range(numner_of_parties):
    ax.annotate(correction_names[i], (first[i], second[i]), fontsize=20)

plt.show()

Answer Section A

Answer Section A:

We make the subpot much bigger to see the parties, and we can see that the left parties are close and also the arabic parties are very close to each other.
Also we can see as we go to the right we can see parties more right-wing views.
Butin the other hand, we can see results that are unexpected such as "Tikva Hadasha" is near to the left parties as "Avoda", "Meretz".

**Question** **3**:

**Section B**

In [ ]:

df_2021_n = normalize(df_2021, axis = 0)

# PCA of the two main components.
df_2021_n_transform = df_2021_n.T

# PCA fit.
pca_a_n = pca.fit_transform(df_2021_n_transform)
fig, ax = plt.subplots(figsize=(12,8))
first_n = pca_a_n[:,0]
second_n = pca_a_n[:,1]

ax.scatter(first_n, second_n)
ax.set_xlabel('first component pc', fontsize=18)
ax.set_ylabel('second component pc', fontsize=18)
ax.set_title('Only two dimension pca', fontsize=18)

numner_of_parties = len(names)
correction_names = [x[::-1] for x in names]

for i in range(numner_of_parties):
    ax.annotate(correction_names[i], (first_n[i],second_n[i]), fontsize=14)
plt.show()

Answer Section B:

We can see better results from section A - first we can see much better the parties that didnt pass the threshold and they are close to each other, We can see the arabic parties close to each other- The centarl-left parties also close to each other.
And the Charidi parties close to each other in the top of the graph.


In [21]:
if  run_in_colab:
    df_socio= pd.read_excel(io.BytesIO(uploaded['eshkol_hevrati_calcali.xlsx']), index_col='name')
else:
    df_socio= pd.read_excel('eshkol_hevrati_calcali.xlsx', index_col='name')

df_2021_raw = df_2021_raw[df_2021_raw.columns[0:-1]]
merged_df = df_2021_raw.join(df_socio.set_index('set_code'), on='סמל ישוב')
merged_df = merged_df.dropna()


**Question** **4**:

**Section A**

In [ ]:
merged_df_names = merged_df[names]

merged_n = normalize(merged_df_names.T, axis=0)

pca_4 = pca.fit_transform(merged_n.T)
first_n = pca_4[:,0]
second_n = pca_4[:,1]
fig, ax = plt.subplots(figsize=(16,12))
sc = ax.scatter(first_n, second_n, c = merged_df['eco_level'], s=4)
plt.colorbar(sc)
ax.set_xlabel('first pca', fontsize=16)
ax.set_ylabel('second pca', fontsize=16)
ax.set_title('2 dimension pca',  fontsize=24)
plt.show()


Answer Section A:

First of all we can see there is three groups in the graph - Charedis, Arabics, Rest.
were we can see the highst and left you go you will find better economic-social, and you will find more support for parties of the left side and if you will go down and left you will find average economic-social but if you will go to the right you will find a bad economic-social were is the Arabics kalpi

**Question** **4**:

**Section B**

In [ ]:
PARTIES = 13
correction_names_passed = [x[::-1] for x in names_passed]
fig, ax = plt.subplots(4, 4, figsize=(30,30))

support_arr = []
for i in range(PARTIES): 
  party = names_passed[i]
  vec_party = merged_df[party]
  support_party = vec_party.div(merged_df['כשרים'])  
  support_arr.append(support_party) 

count = 0
for j in range(4):
  for k in range(4):
    if count < 13:
      sc = ax[j, k].scatter(first_n, second_n, c = support_arr[count], s = 3)
      ax[j, k].set_title(correction_names_passed[count], fontsize=24, color="blue")
      ax[j, k].set_xlabel('first pca', fontsize=16)
      ax[j, k].set_ylabel('second pca', fontsize=16)
    count+=1

plt.colorbar(sc)  
plt.show()

First party - "Po"

We can see that who votes to "Po" have better social-economic and it used to be from very high soceity to middle society - Bur we can't see Kalpi's support "Po", Because we know "Po" is a very big party we can conclude taht there is a correlation between living in high society city to vote "Po".
But also we can see it is not very yellow so it means the support in thode kalpis are not by a large majority

Second party - "Vaadam"

In this party we can see more yellows points that means first that there are citis that have a absolut majority with support to this party.
Also we can conclude that in the cities there's majority to "vaadam" there is low society-economic.
And also we can see that "Vaadam" are far from lots of the kalpi and it means it is kind of minority population.

**Question 1:**

Section A:

In [ ]:
pcs = pd.DataFrame(pca.components_).T
pow_pcs = pcs**2
sqrt_pcs = np.sqrt(pow_pcs.sum())
pcs = pcs.div(sqrt_pcs)
ax = plt.subplot()
first = pcs[0]
second = pcs[1]
ax.bar(np.arange(39),first , color = "black", width = 0.5)
ax.bar(np.arange(39) + 0.5, second, color = "green", width = 0.5)
plt.xticks(np.arange(39))
ax.set_xticklabels(correction_names, rotation='vertical')
plt.show()



**Question 1:**

Section B:

In [ ]:
pca = PCA()
pca_5 = pca.fit_transform(df_2021.T)
var_pca = pca.explained_variance_
plt.bar(np.arange(39), var_pca / sum(var_pca))
plt.xlabel('components')
plt.ylabel('explained variance');

**Question 2:**

Section A:

In [ ]:
total_votes_2021 = parties_votes_percents(df_2021, 0.035)  # total votes for each party
names_2021 = total_votes.keys() # largest parties
total_votes_2020 = parties_votes_percents(df_2020, 0.035)  # total votes for each party
names_2020 = total_votes.keys() # largest parties
# Data with only the largest parties:
df_2021 = df_2021[names]
df_cities = df_2021


Section B:

In [ ]:
#reset
df_2020_ = df_2020_raw[['בזב', 'סמל יישוב', 'בזב']].reset_index()
df_2021_ = df_2021_raw[['בזב', 'סמל יישוב', 'בזב']].reset_index()

join_df = df_2020_index.set_index('סמל ישוב').join(df_2020_index.set_index('סמל ישוב'))
df_2020_raw.index = join_df['שם ישוב']

df_2021_sorted = df_2021_raw.sort_index()
df_2020_sorted = df_2020_raw.sort_index()
df_m = pd.concat([df_2021_sorted, df_2020_sorted])
sum_column = df_m.sum(axis = 1)
df_m = df_m.div(sum_column) #Normlized

We can notice there is an linear growing but also there is point of getting down.

In [ ]:
kosher = pd.concat([df_2021_raw['כשרים'], df_2020_raw['כשרים']])
s_kosher = kosher / max(kosher)
list_2020 = [20] * df_2020_raw.shape[0]
list_2021 = [21] * df_2021_raw.shape[0]
Y = list_2020 + list_2021

pca = PCA(n_components=2)
pca_6 = pca.fit_transform(df_m)
fig, ax = plt.subplots(figsize=(20,14))
first = pca_6[:, 0]
second = pca_6[:, 1]
scatter = ax.scatter(first, second, s = s_kosher, c = Y)
ax.set_xlabel('pca first')
ax.set_ylabel('pca second')


Helper Functions:

In [ ]:
def dist_x_y(d):
    dx = d[0][0] - d[1][0]
    dy = d[0][1] - d[1][1]
    return dx, dy

def calculate_top(df, pca, n):
  pow_distance = (pca[: n] - pca[n : (2*n)])**2
  largest_3 = pow_distance.sum(axis = 1)).nlargest(3).keys()
  return df.index[largest_3]

Section C:

In [ ]:
MORE_THEN = 10000
cities_more_then = kosher[kosher >= MORE_THEN]
for city in cities_more_then.index:
  city = df_m.index.iloc[city]
  d = pca_6[city]
  dx, dy = dist_x_y(d)
  plt.arrow(d[1][0], d[1][1], dx, dy, width = 1/1000, head_length=1/10)
plt.show()

Section D:

In [ ]:
n = df_2021_raw.shape[0]
top_three = calculate_top(df_2021_raw, pca_6, n)
fig, ax = plt.subplots(figsize=(22,16))
first = pca_6[:, 0]
second = pca_6[:, 1]
scatter = ax.scatter(first, second, s = size, c = year)
ax.set_xlabel('pca first')
ax.set_ylabel('pca second')

for city in top_3:
  city = df_m.index.iloc[city]
  d = pca_6[city]
  dx, dy = dist_x_y(d)
  plt.arrow(d[1][0], d[1][1], dx, dy, width = 1/1000, head_length=1/10)
plt.show()

